# HOTRG (cytnx version)

* Author: Pochung Chen
* Last update: 2022/5/9
* Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, and Ω ω.

Two site Hamiltonian: 
$$ \large
  E(s_1, s_2) = -Js_1 s_2
$$

Boltzmann weight on the bond:
$$ \large
  W = e^{-\beta E} = 
  \left[ \begin{array}{cc}
  e^{+\beta J} & e^{-\beta J} \\
  e^{-\beta J} & e^{+\beta J}
  \end{array} \right]
$$

Decompose $W$ as $M M^\dagger$, where
$$ \large
  W = e^{-\beta E} = M M^\dagger,
  M = 
  \left[ \begin{array}{cc}
  +\sqrt{\cosh{\beta J}} & +\sqrt{\sinh{\beta J}} \\
  +\sqrt{\cosh{\beta J}} & -\sqrt{\sinh{\beta J}}
  \end{array} \right]
$$

Numerically, we first perform SVD on $W$ to obtain $M$ and $M^\dagger$:
$$ \large
  W = U S V^\dagger \rightarrow
  M = U \sqrt{S}, M^\dagger = \sqrt{S} U^\dagger.
$$
Note that because $W$ is Hermitian, on has $V=U$.

$$ \large
  T_{ijkl} = \sum_\alpha
  M_{\alpha, i} M_{\alpha, j} M_{\alpha, k} M_{\alpha, l}
$$

In [1]:
import numpy as np
import cytnx

In [2]:
def ut_print(ut):
    ut.print_diagram()
    print(ut.get_block().numpy())

## Constrauct $M$

### numpy version

### Boltzmann weight

In [3]:
def W0_(T):
    W0 = np.array([[np.exp(+1/T),np.exp(-1/T)],
                   [np.exp(-1/T),np.exp(+1/T)]])
    return W0

### $M$, $M^\dagger$ from SVD

In [4]:
# W=USVd
# print('[W0]=\n', W0_(1.0))
U, S, Vd = np.linalg.svd(W0_(1.0))
# print('[S]=\n', S)
M = U @ np.diag(np.sqrt(S))
Md = np.diag(np.sqrt(S)) @ Vd
print('[M]=\n', M)
print('[M]d=\n', Md)
print('[M][Md]-[W]=\n', M @ Md - W0_(1.0))

[M]=
 [[-1.24220797 -1.08406697]
 [-1.24220797  1.08406697]]
[M]d=
 [[-1.24220797 -1.24220797]
 [-1.08406697  1.08406697]]
[M][Md]-[W]=
 [[-8.8817842e-16 -4.4408921e-16]
 [ 0.0000000e+00 -4.4408921e-16]]


### $M$, $M^\dagger$ from exact formula

In [5]:
def M0_(T):
    M0 = np.array([[+np.sqrt(np.cosh(1/T)), +np.sqrt(np.sinh(1/T))],
                   [+np.sqrt(np.cosh(1/T)), -np.sqrt(np.sinh(1/T))]])
    return M0

a_M = M0_(1)
print('[M]=\n', a_M)
print('[M][Md]=\n', a_M @ a_M.transpose())
print('[M][Md]-[W]=\n', a_M @ a_M.transpose()-W0_(1.0))

[M]=
 [[ 1.24220797  1.08406697]
 [ 1.24220797 -1.08406697]]
[M][Md]=
 [[2.71828183 0.36787944]
 [0.36787944 2.71828183]]
[M][Md]-[W]=
 [[ 0.00000000e+00 -3.88578059e-16]
 [-3.88578059e-16  0.00000000e+00]]


### cytnx version (SVD)
* cytnx.from_numpy()
* cytnx.linalg.Svd()
* cytnx.linalg.Pow()
* cytnx.linalg.Diag()
* cytnx.UniTensor()

In [6]:
def M_(T, h):
    W = np.array([[np.exp(1/T), np.exp(-1/T)],
                  [np.exp(-1/T), np.exp(1/T)]])
    W = cytnx.from_numpy(W)
    S, U, Vd = W.Svd()
    M = U @ cytnx.linalg.Diag(S.Pow(0.5))
    Md = cytnx.linalg.Diag(S.Pow(0.5)) @ Vd
    M = cytnx.UniTensor(M, rowrank=1)
    M.set_name('M')
    Md = cytnx.UniTensor(Md, rowrank=1)
    Md.set_name('M†')    
    return M, Md

ut_M, ut_Md = M_(1, 0)
ut_print(ut_M)
ut_print(ut_Md)

-----------------------
tensor Name : M
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
[[-1.24220797 -1.08406697]
 [-1.24220797  1.08406697]]
-----------------------
tensor Name : M†
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
[[-1.24220797 -1.24220797]
 [-1.08406697  1.08406697]]


### Check [M][M†]=[W]
* If we use cytnx.Contract(), we need to re-label.

In [7]:
ut_print(cytnx.Contract(ut_M, ut_Md))

-----------------------
tensor Name : 
tensor Rank : 0
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           \             /     
            -------------      
[5.41155508]


#### re-label before contract

In [8]:
ut_Md.set_labels((1,2))
ut_print(ut_Md)

-----------------------
tensor Name : M†
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     1 ____| 2         2 |____ 2  
           \             /     
            -------------      
[[-1.24220797 -1.24220797]
 [-1.08406697  1.08406697]]


In [9]:
print(W0_(1.0))
ut_print(cytnx.Contract(ut_M, ut_Md))

[[2.71828183 0.36787944]
 [0.36787944 2.71828183]]
-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           \             /     
            -------------      
[[2.71828183 0.36787944]
 [0.36787944 2.71828183]]


## Construct $T_{bare}$

### Construct $\delta$ tensor

In [10]:
def delta_():
    delta = cytnx.zeros([2,2,2,2])
    delta[0,0,0,0]= 1.0
    delta[1,1,1,1]= 1.0
    delta = cytnx.UniTensor(delta, rowrank=2)
    delta.set_name('ẟ')
    return delta

ut_ẟ = delta_()
# print(ut_ẟ.get_block().numpy())
# ut_ẟ.print_diagram()
ut_print(ut_ẟ)

-----------------------
tensor Name : ẟ
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[1. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 1.]]]]


### Construct $T_{bare}$ tensor

In [11]:
ut_M, ut_Md = M_(1, 0)
ut_ẟ = delta_()
# ut_print(ut_ẟ)
T_bare_net = cytnx.Network('Networks/T_bare.net')
print(T_bare_net)
T_bare_net.PutUniTensors(['ẟ', 'M0.d', 'M1.d', 'M2', 'M3'], [ut_ẟ, ut_Md, ut_Md, ut_M, ut_M])
print(T_bare_net)
T_bare = T_bare_net.Launch(optimal=True)
T_bare.set_name('T_bare')
ut_print(T_bare)

==== Network ====
[x] ẟ : 100 101 ; 102 103 
[x] M0.d : 0 ; 100 
[x] M1.d : 1 ; 101 
[x] M2 : 102 ; 2 
[x] M3 : 103 ; 3 
TOUT : 0 1 ; 2 3 
ORDER : 

==== Network ====
[o] ẟ : 100 101 ; 102 103 
[o] M0.d : 0 ; 100 
[o] M1.d : 1 ; 101 
[o] M2 : 102 ; 2 
[o] M3 : 103 ; 3 
TOUT : 0 1 ; 2 3 
ORDER : 

-----------------------
tensor Name : T_bare
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[ 4.76219569e+00  2.32739908e-15]
   [ 2.32739908e-15  3.62686041e+00]]

  [[ 2.05157350e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.01055929e-15]]]


 [[[ 2.16101918e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.21683176e-15]]

  [[ 3.62686041e+00 -2.77274255e-15]
   [-2.77274255e-15  2.76219569e+00]]]]


In [12]:
ut_M, ut_Md = M_(1, 0)
ut_ẟ = delta_()
#                       0
#                 ┌─────┘
#                 │ ┏━━━╳━━━┓ 
#                 └─┨d MT0 d┠─┐
#                   ┗━━━━━━━┛ │                                    0
#                 ┌────100────┘                              ┌─────┘
#                 │ ┏━━━╳━━━┓                                │ ┏━━━╳━━━━┓
#    ┏━━━╳━━━┓    └─┨d      ┃      ┏━━━╳━━━┓                 └─┨d       ┃
# 1──┨d MT1 d┠─101──┨d  ẟ  d┠──102─┨d  M2 d┠──2    =     1 ────┨d TOUT d┠──── 2
#    ┗━━━━━━━┛      ┃      d┠─┐    ┗━━━━━━━┛                   ┃       d┠─┐
#                   ┗━━━━━━━┛ │                                ┗━━━━━━━━┛ │
#                 ┌────103────┘                                    ┌──────┘              
#                 │ ┏━━━╳━━━┓                                      3
#                 └─┨d  M3 d┠─┐
#                   ┗━━━━━━━┛ │
#                       ┌─────┘
#                       3

T_bare_net = cytnx.Network()
T_bare_net.FromString([
'ẟ: 100,101;102,103',
'M0.d: 0;100',
'M1.d: 1;101',
'M2  : 102;2',
'M3  : 103;3',
'TOUT: 0,1;2,3',
'ORDER:'
])

print(T_bare_net)
T_bare_net.PutUniTensors(['ẟ', 'M0.d', 'M1.d', 'M2', 'M3'], [ut_ẟ, ut_Md, ut_Md, ut_M, ut_M])
print(T_bare_net)
T_bare = T_bare_net.Launch(optimal=True)
T_bare.set_name('T_bare')
ut_print(T_bare)

==== Network ====
[x] ẟ : 100 101 ; 102 103 
[x] M0.d : 0 ; 100 
[x] M1.d : 1 ; 101 
[x] M2 : 102 ; 2 
[x] M3 : 103 ; 3 
TOUT : 0 1 ; 2 3 
ORDER : 

==== Network ====
[o] ẟ : 100 101 ; 102 103 
[o] M0.d : 0 ; 100 
[o] M1.d : 1 ; 101 
[o] M2 : 102 ; 2 
[o] M3 : 103 ; 3 
TOUT : 0 1 ; 2 3 
ORDER : 

-----------------------
tensor Name : T_bare
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[ 4.76219569e+00  2.32739908e-15]
   [ 2.32739908e-15  3.62686041e+00]]

  [[ 2.05157350e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.01055929e-15]]]


 [[[ 2.16101918e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.21683176e-15]]

  [[ 3.62686041e+00 -2.77274255e-15]
   [-2.77274255e-15  2.76219569e+00]]]]


### Construct $T$ using numpy

In [13]:
a_M0 = M0_(1)
a_T = np.zeros((2,2,2,2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                a_T[i,j,k,l]=a_M0[0,i]*a_M0[0,j]*a_M0[0,k]*a_M0[0,l]+a_M0[1,i]*a_M0[1,j]*a_M0[1,k]*a_M0[1,l]
a_T 

array([[[[4.76219569, 0.        ],
         [0.        , 3.62686041]],

        [[0.        , 3.62686041],
         [3.62686041, 0.        ]]],


       [[[0.        , 3.62686041],
         [3.62686041, 0.        ]],

        [[3.62686041, 0.        ],
         [0.        , 2.76219569]]]])

### Merge two $T_{bare}$ in y-direction

In [14]:
TupTdn_net = cytnx.Network('Networks/TupTdn.net')
print(TupTdn_net)
TupTdn_net.PutUniTensors(['Tup', 'Tdn'], [T_bare, T_bare])
print(TupTdn_net)
TupTdn = TupTdn_net.Launch(optimal=True)
TupTdn.set_name('Tupdn')
TupTdn.print_diagram()
# ut_print(TupTdn)
TupTdn.combineBonds([1,2])
TupTdn.combineBonds([3,4])
TupTdn.print_diagram()

==== Network ====
[x] Tup : 0 1 ; 3 -1 
[x] Tdn : -1 2 ; 4 5 
TOUT : 0 1 2 ; 3 4 5 
ORDER : 

==== Network ====
[o] Tup : 0 1 ; 3 -1 
[o] Tdn : -1 2 ; 4 5 
TOUT : 0 1 2 ; 3 4 5 
ORDER : 

-----------------------
tensor Name : Tupdn
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 3  
           |             |     
     1 ____| 2         2 |____ 4  
           |             |     
     2 ____| 2         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : Tupdn
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 5  
           \             /     
            -------------      


In [15]:
#           0
#     ┌─────┘
#     │ ┏━━━╳━━┓                          0
#     └─┨      ┃                    ┌─────┘
#  1 ───┨  Tup ┠───3                │ ┏━━━╳━━┓
#       ┃      ┠─┐                  └─┨      ┃
#       ┗━━━━━━┛ │               1────┨      ┃
#     ┌─── -1 ───┘       -->     2────┨ TOUT ┠────3
#     │ ┏━━━╳━━┓                      ┃      ┠────4
#     └─┨      ┃                      ┃      ┠─┐
#  2────┨  Tdn ┠────4                 ┗━━━━━━┛ │
#       ┃      ┠─┐                        ┌────┘
#       ┗━━━━━━┛ │                        5
#           ┌────┘
#           5

TupTdn_net = cytnx.Network()
TupTdn_net.FromString([
'Tup: 0,1;3,-1',
'Tdn: -1,2;4,5',
'TOUT: 0,1,2; 3,4,5',
'ORDER:'
])
print(TupTdn_net)
TupTdn_net.PutUniTensors(['Tup', 'Tdn'], [T_bare, T_bare])
print(TupTdn_net)
TupTdn = TupTdn_net.Launch(optimal=True)
TupTdn.set_name('TupTdn')
TupTdn.print_diagram()
# ut_print(TupTdn)
TupTdn.combineBonds([1,2])
TupTdn.combineBonds([3,4])
TupTdn.print_diagram()

==== Network ====
[x] Tup : 0 1 ; 3 -1 
[x] Tdn : -1 2 ; 4 5 
TOUT : 0 1 2 ; 3 4 5 
ORDER : 

==== Network ====
[o] Tup : 0 1 ; 3 -1 
[o] Tdn : -1 2 ; 4 5 
TOUT : 0 1 2 ; 3 4 5 
ORDER : 

-----------------------
tensor Name : TupTdn
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 3  
           |             |     
     1 ____| 2         2 |____ 4  
           |             |     
     2 ____| 2         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : TupTdn
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 5  
           \             /     
            -------------      


### Construct $(1\times 2)$ transfer matrix $\mathcal{T}_{1x2}$

In [16]:
TM = TupTdn.Trace(0, 5, by_label=True)
TupTdn.print_diagram()
# TM.print_diagram()
ut_print(TM)

-----------------------
tensor Name : TupTdn
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : TupTdn
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     1 ____| 4         4 |____ 3  
           \             /     
            -------------      
[[ 3.58326242e+01  1.56953966e-14  1.56953966e-14  2.63082328e+01]
 [ 1.59925108e-14  2.72899172e+01  2.63082328e+01 -2.14195957e-14]
 [ 1.59925108e-14  2.63082328e+01  2.72899172e+01 -2.14195957e-14]
 [ 2.63082328e+01 -2.11933153e-14 -2.11933153e-14  2.07838415e+01]]


In [17]:
TM = TupTdn.Trace(0, 3)
TupTdn.print_diagram()
# TM.print_diagram()
ut_print(TM)

-----------------------
tensor Name : TupTdn
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : TupTdn
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     1 ____| 4         4 |____ 3  
           \             /     
            -------------      
[[ 3.58326242e+01  1.56953966e-14  1.56953966e-14  2.63082328e+01]
 [ 1.59925108e-14  2.72899172e+01  2.63082328e+01 -2.14195957e-14]
 [ 1.59925108e-14  2.63082328e+01  2.72899172e+01 -2.14195957e-14]
 [ 2.63082328e+01 -2.11933153e-14 -2.11933153e-14  2.07838415e+01]]


In [18]:
λ, _ = TM.get_block().Eigh()
λ = λ.numpy()
print(λ)
λ = λ[::-1]
print(λ)

[ 0.94512664  0.98168436 53.59815003 55.67133903]
[55.67133903 53.59815003  0.98168436  0.94512664]


In [19]:
aTM = TM.get_block()
λ = np.linalg.eigvalsh(aTM.numpy())
print(λ)
λ = λ[::-1]
print(λ)

[ 0.94512664  0.98168436 53.59815003 55.67133903]
[55.67133903 53.59815003  0.98168436  0.94512664]


In [20]:
E = -np.log(λ)
print(E)

[-4.01946545 -3.98151455  0.01848545  0.05643635]


## Refactor

In [21]:
T = {}
TM = {}

In [22]:
#           0
#     ┌─────┘
#     │ ┏━━━╳━━┓                          0
#     └─┨      ┃                    ┌─────┘
#  1 ───┨  Tup ┠───3                │ ┏━━━╳━━┓
#       ┃      ┠─┐                  └─┨      ┃
#       ┗━━━━━━┛ │               1────┨      ┃
#     ┌─── -1 ───┘       -->     2────┨ TOUT ┠────3
#     │ ┏━━━╳━━┓                      ┃      ┠────4
#     └─┨      ┃                      ┃      ┠─┐
#  2────┨  Tdn ┠────4                 ┗━━━━━━┛ │
#       ┃      ┠─┐                        ┌────┘
#       ┗━━━━━━┛ │                        5
#           ┌────┘
#           5
def merge_y(Tup, Tdn):
    TupTdn_net = cytnx.Network()
    TupTdn_net.FromString([
    'Tup: 0,1;3,-1',
    'Tdn: -1,2;4,5',
    'TOUT: 0,1,2; 3,4,5',
    'ORDER:'
    ])
    TupTdn_net.PutUniTensors(['Tup', 'Tdn'], [Tup, Tdn])
    return TupTdn_net.Launch(optimal=True)

### Construct $(1\times 2)$ transfer matrix $\mathcal{T}_{1x2}$

In [23]:
T[(1,2)] = merge_y(T_bare, T_bare)
T[(1,2)].set_name('T_{1,2}')
T[(1,2)].print_diagram()
T[(1,2)].combineBonds([1,2])
T[(1,2)].combineBonds([3,4])
T[(1,2)].print_diagram()

TM[(1,2)] = T[(1,2)].Trace(0, 3)
λ, _ = TM[(1,2)].get_block().Eigh()
λ = λ.numpy()
print(λ)
λ = λ[::-1]
print(λ)
E = -np.log(λ)
print(E/2)

-----------------------
tensor Name : T_{1,2}
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 3  
           |             |     
     1 ____| 2         2 |____ 4  
           |             |     
     2 ____| 2         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : T_{1,2}
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 5  
           \             /     
            -------------      
[ 0.94512664  0.98168436 53.59815003 55.67133903]
[55.67133903 53.59815003  0.98168436  0.94512664]
[-2.00973273 -1.99075728  0.00924272  0.02821817]


### Construct $(1\times 3)$ transfer matrix $\mathcal{T}_{1x3}$

In [24]:
T[(1,3)] = merge_y(T[(1,2)], T_bare)
T[(1,3)].set_name('T_{1,3}')
T[(1,3)].print_diagram()
T[(1,3)].combineBonds([1,2])
T[(1,3)].combineBonds([3,4])
T[(1,3)].print_diagram()

TM[(1,3)] = T[(1,3)].Trace(0, 3)
λ, _ = TM[(1,3)].get_block().Eigh()
λ = λ.numpy()
print(λ)
λ = λ[::-1]
print(λ)
E = -np.log(λ)
print(E/3)

-----------------------
tensor Name : T_{1,3}
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         4 |____ 3  
           |             |     
     1 ____| 4         2 |____ 4  
           |             |     
     2 ____| 2         2 |____ 5  
           \             /     
            -------------      
-----------------------
tensor Name : T_{1,3}
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         8 |____ 3  
           |             |     
     1 ____| 8         2 |____ 5  
           \             /     
            -------------      
[  5.33302166   6.27203645   6.27203645   8.23540518   8.23540518
   9.14392425 402.73718286 404.96291155]
[404.96291155 402.73718286   9.14392425   8.23540518   8.23540518
   6.27203645   6.27203645   5.3